# EDA

In [ ]:
# imports
import numpy as np
import pandas as pd
import glob
from pydicom.data import get_testdata_file
from pydicom import dcmread
import matplotlib.pyplot as plt
from tqdm import trange

In [ ]:
# read in data
train_df = pd.read_csv('/kaggle/input/osic-pulmonary-fibrosis-progression/train.csv')
test_df = pd.read_csv('/kaggle/input/osic-pulmonary-fibrosis-progression/test.csv')

In [ ]:
# find total number of unique patients
train_df['Patient'].nunique()

In [ ]:
# finding number of patients that are male and female
print(test_df['Sex'].unique())
print(train_df[train_df['Sex'] == 'Female'].shape)
print(train_df[train_df['Sex'] == 'Male'].shape)

In [ ]:
# getting list of paths to all subjects folders
subject_paths = glob.glob('/kaggle/input/osic-pulmonary-fibrosis-progression/train/*')

In [ ]:
# getting list of all dicom paths
dicom_paths = []
for subject in subject_paths:
    subject_dicoms = glob.glob(subject + '/*')
    dicom_paths.append(subject_dicoms)

In [ ]:
# reading in a particular dicom path
ds = dcmread(dicom_paths[100][13])

In [ ]:
# displaying the pixel array in the dicom
plt.figure(figsize = (7, 7))
plt.imshow(ds.pixel_array, cmap="plasma")
plt.axis('off');

In [ ]:
# find max value in the pixel array
ds.pixel_array.max()

#  Model Training

In [ ]:
# checking which GPU was allocated
!nvidia-smi

## Libraries Utilized
* numpy - used for arrays and fast random selection
* pandas - used for reading in csv file and accessing row data for generating model inputs and targets
* glob - used for finding all dicom files belonging to a particular patient (finds dicom files in patient folder)
* matplotlib - used for visualizing training and validation loss curves
* pydicom - used as an interface to read in dicom information and pixel values
* cv2 - used to resize images to a smaller and consistent shape
* scikit-learn - used for ```train_test_split``` and for r^2 and RMSE calculation
* rich - used for a progress bar to visualize model training runtime
* pytorch - used for model creation, training, and data preparation
* efficientnet_pytorch - used for pretrained EfficientNet models for use in transfer learning

In [ ]:
!pip install rich efficientnet_pytorch

In [ ]:
# imports
import math
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from pydicom import dcmread
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from rich.progress import track

In [ ]:
# Torch-specific imports
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from efficientnet_pytorch import EfficientNet

In [ ]:
# Function to read in dicom pixel values from file, and to reshape and resize to smaller size - 224x224
def get_img(path):
    # read in file with pydicom
    d = dcmread(path)
    # rescale pixel intensities to match actual values
    pxls = (d.pixel_array - d.RescaleIntercept) / (d.RescaleSlope * 1000)
    # resize pixel intensities to a 224 by 224 array
    return cv2.resize(pxls, (224, 224))

## PyTorch Dataset Class
### ```__init__``` function:
* Variables:
    * ```self.mode``` to tell function where to find dicom images for particular subject (train or test folder)
    * ```self.df``` specifies the dataframe containing train, validation, or test patients/FVC measurements
    * ```self.transforms``` specifies particular transforms applied on image - here, the image is only converted to a pytorch tensor

### ```__len__``` function:
 * Function that tells PyTorch the size of the dataset being used
 * set to ```len(self.df)``` because the dataframe contains the total number of pids and each image/target is selected once per patient

### ```__getitem__``` function:
* particular data point at specified idx selected
* ```pid``` stores Patient ID for the particular datapoint
* ```label``` stores FVC measurement for a particular Patient ID
* ```img``` contains a random image chosen from all patient dicoms after being transformed
* Tensors of type double are returned (model requires double data type)

In [ ]:
class OsicDataset(Dataset):
    def __init__(self, df, mode = 'train'):
        self.mode = mode
        self.df = df
        self.transforms = transforms.Compose([
            transforms.ToTensor(),
#             transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # particular row of dataset
        row = self.df.iloc[idx,:]
        # patient id for certain row
        pid = row['Patient']
        # fvc measurement for particular patient
        label = row['FVC']
        # list of all dicom file paths within patient directory is obtained with ```glob.glob```
        img_ids = glob.glob('/kaggle/input/osic-pulmonary-fibrosis-progression/' + self.mode + '/' + pid + '/*')
        # random dicom file path is selected and transformed through the ```get_img``` function
        img = get_img(np.random.choice(img_ids))
        # Image is converted to PyTorch tensor
        img = self.transforms(img)
        
        # Tensors are changed to ```torch.double``` datatype
        # returns dictionary of image and target
        return {
            'img': img.type(torch.DoubleTensor),
            'target': torch.tensor(label, dtype=torch.double),
        }

## Model Specification
Two models are created:
* CustomModel
    * custom model created using prior code
* EffModel
    * model that makes use of pretrained neural networks: created with help from https://www.kaggle.com/noelmat/pytorch-efficientnet-with-better-dataloaders
    * ```eff_name``` specifies which model (from b0-b7) is to be used as the pretrained model - here, ```b1``` is used

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,kernel_size=(4,4),padding=1,stride=2)
        self.max_pool = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(4, 4), padding=1, stride=2)
        self.bn1 = nn.BatchNorm2d(64)
        self.dropout = nn.Dropout(0.15)
        self.conv3 = nn.Conv2d(64, 4, kernel_size=(4, 4))
        self.bn2 = nn.BatchNorm2d(4)
        self.ada_pool = nn.AdaptiveMaxPool2d((28, 28))
        self.fc1 = nn.Linear(28 * 28 * 4, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # first convolutional layer with convs of size 4 and 32 channels
        x = self.conv1(x)
        # thresholded with relu
        x = self.relu(x)
        # max pooling to reduce image size
        x = self.max_pool(x)
        # second convolutional layer with convs of size 4 and 64 channels
        x = self.conv2(x)
        x = self.relu(x)
        # batch normalization used to normalize outputs so far
        x = self.bn1(x)
        # max pooling to reduce image size
        x = self.max_pool(x)
        # dropout to reduce model overfitting
        x = self.dropout(x)
        # third convolutional layer with convs of size 4 and 4 channels
        x = self.conv3(x)
        x = self.relu(x)
        x = self.bn2(x)
        # adaptive pooling to resize images to 228 by 228
        x = self.ada_pool(x)
        
        # flatten image to be passed through fully connected layers
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        return x

In [ ]:
class EffModel(nn.Module):
    def __init__(self, eff_name='b0'):
        super().__init__()
        self.conv = nn.Conv2d(1, 3, kernel_size=3, padding=1, stride=2)
        self.bn = nn.BatchNorm2d(3)
        self.model = EfficientNet.from_pretrained(f'efficientnet-{eff_name}')
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        # one initial convolution layer with convs of size 3 and 3 channels
        x = self.conv(x)
        # batch normalization
        x = self.bn(x)
        x = self.relu(x)
        
        # pass through pretrained model
        x = self.model(x)
        # image passed through two additional fully connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        return x

In [ ]:
# b1 model is being used
net = EffModel(eff_name='b1')
# model set to type torch.double
net.double();

## Train Function
* Created with use of previous mimic_bert code
* Allows GPU training
* Mean Squared Error used as loss function
* Adam used as optimizer function
* MSE loss, RMSE, and R^2 calculated during training and validation
* RMSE and R^2 calculated during train validation, and test
* Loss curve of training and validation loss is plotted

In [ ]:
def train(model, train_dataloader, valid_dataloader, test_dataloader, epochs):
    # have torch recognize GPU if it exists, otherwise use CPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # load model onto the device
    model.to(device)
    
    # set loss to MSELoss and optimizer to Adam
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    
    print("start of training loop")
    
    # creating lists to store training and validation losses
    train_losses = []
    val_losses = []
    
    # iterating model training for each epoch
    for epoch in track(range(epochs), description='Training...'):
        model.train()
        train_loss = 0
        train_rmse = 0
        train_rsq = 0
        count = 0
        
        for batch in train_dataloader:
            # set to zero so gradients are not accumulated
            optimizer.zero_grad()
            
            # load images and targets onto device
            imgs = batch['img'].to(device)
            targets = batch['target'].to(device)
            
            # pass images through model
            out = model(imgs)
            
            # calculate loss and backpropagate through network
            loss = criterion(out, targets.view(-1, 1))
            loss.backward()
            optimizer.step()
            
            # add train_loss for particular training step to epoch training loss
            train_loss += loss.item() * targets.size(0)
            
            # get predicted and actual values for targets
            predicted = out.cpu().detach().numpy()
            actual = targets.cpu().detach().numpy()
            
            # use predicted and actual values for target to calculate rmse and r^2 score
            train_rmse += mean_squared_error(actual, predicted, squared=False)
            train_rsq  += r2_score(actual, predicted)
            count += 1
        
        # divide train_loss by number of training steps in order to get epoch train_loss
        train_loss /= len(train_dataloader.sampler)
        train_losses.append(train_loss)
        
        # divide rmse and r^2 to get epoch rmse and r^2 score
        train_rmse /= count
        train_rsq /= count
        
        # Validation
        model.eval()
        val_loss = 0
        val_rmse = 0
        val_rsq = 0
        count = 0
        
        for batch in val_dataloader:
            optimizer.zero_grad()
            
            imgs = batch['img'].to(device)
            targets = batch['target'].to(device)
            
            out = model(imgs)
            loss = criterion(out, targets.view(-1, 1))
            
            val_loss += loss.item() * targets.size(0)
            
            predicted = out.cpu().detach().numpy()
            actual = targets.cpu().detach().numpy()
            
            val_rmse += mean_squared_error(actual, predicted, squared=False)
            val_rsq  += r2_score(actual, predicted)
            count += 1
        
        val_loss /= len(val_dataloader.sampler)
        val_losses.append(val_loss)
        
        val_rmse /= count
        val_rsq /= count
        
        # print metrics
        print(
            "\n",
            "\n",
            f"Epoch {epoch+1}/{epochs}:\n",
            f"Train loss: {train_loss:.3f}...\n",
            f"Valid loss: {val_loss:.3f}...\n",
            "\n",
            f"Train RMSE: {train_rmse:.3f}...\n",
            f"Valid RMSE: {val_rmse:.3f}...\n",
            "\n",
            f"Train R^2: {train_rsq}...\n",
            f"Valid R^2: {val_rsq}...\n",
        )
    
    
    # Test
    model.eval()
    
    test_rmse = 0
    test_rsq = 0
    count = 0

    for batch in test_dataloader:

        imgs = batch['img'].to(device)
        targets = batch['target'].to(device)

        out = model(imgs)
    
        predicted = out.cpu().detach().numpy()
        actual = targets.cpu().detach().numpy()
        
        test_rmse += mean_squared_error(actual, predicted, squared=False)
        test_rsq  += r2_score(actual, predicted)
        count += 1

    test_rmse /= count
    test_rsq /= count

    print(
        "\n",
        "\n",
        "\n",
        "Training Finished!\n",
        f"Test RMSE: {test_rmse:.3f}...\n",
        f"Test R^2: {test_rsq:.3f}...\n\n",
        f"Predictions: {predicted}\n",
        f"Actual: {actual} \n\n",
    )
    
    # display loss curves
    print('Loss Curves: ')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    # plot train losses per epoch
    plt.plot(list(range(epochs)), train_losses, label='train')
    # plot validation losses per epoch
    plt.plot(list(range(epochs)), val_losses, label='valid')
    plt.legend()
    plt.show()

## Preprocessing

* provided csv files are read into ```train_df``` and ```test_df``` respectively
* subjects with dicom files that cause errors due to being stored in a different compressed format were not included
* Provided train dataframe was partitioned into train and validation with in a ratio of 0.7:0.3

In [ ]:
train_df = pd.read_csv('/kaggle/input/osic-pulmonary-fibrosis-progression/train.csv') 
test_df  = pd.read_csv('/kaggle/input/osic-pulmonary-fibrosis-progression/test.csv')

In [ ]:
train_df = train_df.drop(np.nonzero(np.array(train_df['Patient'] == 'ID00011637202177653955184',dtype=float))[0], axis=0).reset_index(drop=True)
train_df = train_df.drop(np.nonzero(np.array(train_df['Patient'] == 'ID00052637202186188008618',dtype=float))[0], axis=0).reset_index(drop=True)

In [ ]:
train_df, val_df = train_test_split(train_df, test_size=0.3)

## Data Loading
* ```train_df```,```val_df```, and ```test_df``` are passed into train, validation, and test instances of the ```OsicDataset``` class
* PyTorch DataLoaders are created that prepare data in batches of 32 and randomly shuffle data

In [ ]:
train_dataset = OsicDataset(train_df)
val_dataset = OsicDataset(val_df)
test_dataset = OsicDataset(test_df, mode='test')

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

## Training Loop
* ```net``` (convolutional neural network chosen) is passed into train function
* train, validation, and test dataloaders are passed into train function
* 10 epochs of training are specified

In [ ]:
train(
    net,
    train_dataloader,
    val_dataloader,
    test_dataloader,
    epochs=10
)

## Cleanup
* gc package used for garbage collection to free RAM memory
* ```torch.cuda.empty_cache()``` used to free GPU VRAM memory

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

# Combining both a CNN and RNN to include Tabular Data

## Preprocessing

In [ ]:
# read in data
train_df = pd.read_csv('/kaggle/input/osic-pulmonary-fibrosis-progression/train.csv')
test_df = pd.read_csv('/kaggle/input/osic-pulmonary-fibrosis-progression/test.csv')

In [ ]:
# dropping patients who have dicoms that raise errors when trying to open with pydicom
train_df = train_df.drop(np.nonzero(np.array(train_df['Patient'] == 'ID00011637202177653955184',dtype=float))[0], axis=0).reset_index(drop=True)
train_df = train_df.drop(np.nonzero(np.array(train_df['Patient'] == 'ID00052637202186188008618',dtype=float))[0], axis=0).reset_index(drop=True)

In [ ]:
print(train_df.columns)
print(train_df['SmokingStatus'].unique())

In [ ]:
# dropping Percent and rearranging columns
cols = ['Patient', 'Age', 'Sex', 'SmokingStatus', 'Weeks', 'FVC']
train_df = train_df[cols]
test_df = test_df[cols]

In [ ]:
test_df

In [ ]:
# label encoding Sex and SmokingStatus
train_df['Sex'] = train_df['Sex'].astype('category').cat.codes
train_df['SmokingStatus'] = train_df['SmokingStatus'].astype('category').cat.codes

test_df['Sex'] = test_df['Sex'].astype('category').cat.codes
test_df['SmokingStatus'] = test_df['SmokingStatus'].astype('category').cat.codes

In [ ]:
print(train_df.info())
print(test_df.info())

In [ ]:
# taken from https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    # get number of variables to shift
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # shift feature data backward in order to use past n_in time points to predict next fvc values
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # create target column(s) - n_out time points to be predicted
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    
    # concatenating all shift columns together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # drop nan values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# get dfs for each patient
train_split = [subj_data for _, subj_data in train_df.groupby('Patient', as_index=False)]
# get values of each df in train_split
train_vals = [df.values for df in train_split]
# apply series to supervised function on the values
train_time_series = [series_to_supervised(val) for val in train_vals]
# sort values in time series by week just in case measurements in the original dataset are not in order
train_time_series = [ts.sort_values(by='var5(t-1)') for ts in train_time_series]

In [ ]:
# concatenate list of time series for each patient
train_ts = pd.concat(train_time_series)

In [ ]:
# only predicting next fvc measurement, not other variables
drop_idxs = list(range(6, train_ts.shape[1] - 1))
train_ts = train_ts.drop(train_ts.columns[drop_idxs], axis=1)
test_ts = test_df

In [ ]:
train_ts

In [ ]:
# label encoding the patient column
train_ts['Patient'] = train_ts['var1(t-1)'].astype('category').cat.codes

In [ ]:
# rearrange columns to make the df look nicer
train_ts = train_ts[['var1(t-1)', 'Patient', 'var2(t-1)', 'var3(t-1)', 'var4(t-1)', 'var5(t-1)', 'var6(t-1)', 'var6(t)']]

In [ ]:
train_ts

In [ ]:
test_ts

In [ ]:
submission = pd.read_csv('/kaggle/input/osic-pulmonary-fibrosis-progression/sample_submission.csv')
submission = pd.DataFrame(submission.Patient_Week.str.split('_',1).tolist(),
                                 columns = ['Patient','Weeks'])

In [ ]:
submission

# CombinedDataset Class
* Relatively similar to the ```OsicDataset``` but includes other feature data to be used in time series prediction (smoking status, gender, etc.)

In [ ]:
class CombinedDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.transforms = transforms.Compose([
            transforms.ToTensor(),
#             transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # particular row of dataset
        row = self.df.iloc[idx,:]

        # patient id for certain row
        pid = row['var1(t-1)']
        # fvc measurement for particular patient
        label = row['var6(t)']
        
        ## Image Processing
        
        # list of all dicom file paths within patient directory is obtained with ```glob.glob```
        img_ids = glob.glob('/kaggle/input/osic-pulmonary-fibrosis-progression/train/' + pid + '/*')
        # random dicom file path is selected and transformed through the ```get_img``` function
        img = get_img(np.random.choice(img_ids))
        # Image is converted to PyTorch tensor
        img = self.transforms(img)
        
        ## Time Series Processing
        # don't include pid char string or label at the end
        ts_data = np.array(list(row[1:-1].values))
        ts_data = ts_data.reshape(1, 6)
        # Tensors are changed to ```torch.double``` datatype
        # returns dictionary of image and target
        return {
            'img': img.type(torch.DoubleTensor),
            'ts': torch.tensor(ts_data, dtype=torch.double),
            'target': torch.tensor(label, dtype=torch.double),
        }

# CombinedModel Class
* similar to ```EffModel``` but includes the time series data through the use of an LSTM
* hidden_size of the LSTM can be changed
* output tensors from RNN and CNN are concatenated and passed through a last fully connected layer to create output

In [ ]:
class CombinedModel(nn.Module):
    def __init__(self, hidden_size, eff_name='b1'):
        super().__init__()
        
        ## img layers
        self.conv = nn.Conv2d(1, 3, kernel_size=3, padding=1, stride=2)
        self.bn = nn.BatchNorm2d(3)
        self.model = EfficientNet.from_pretrained(f'efficientnet-{eff_name}')
        self.fc1 = nn.Linear(1000, 500)
        
        ## ts layer
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size=6,hidden_size=self.hidden_size, num_layers=2, dropout=0.4, batch_first=True)
        
        self.output = nn.Linear(500 + hidden_size, 1)
        self.relu = nn.ReLU()
        
    def forward(self, img, ts):
        # apply conv to increase channels to 3
        img = self.conv(img)
        # batch norm to 
        img = self.bn(img)
        img = self.relu(img)
        # pass through efficientnet model
        img = self.model(img)
        # fc layer to condense outputs down to 500
        img = self.fc1(img)
        img = self.relu(img)
        
        # run rnn on time series data
        ts, _ = self.rnn(ts)
        
        # concatenate image features with time series features
        x = torch.cat([img, ts.view(-1,self.hidden_size)], dim=1)
        # pass through last fc layer
        return self.output(x)

In [ ]:
def train_loop(model, train_dataloader, val_dataloader, epochs):
    
    # have torch recognize GPU if it exists, otherwise use CPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # load model onto the device
    model.to(device)
    
    # set loss to MSELoss and optimizer to Adam
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.L1Loss()
    
    print("start of training loop")
    
    # creating lists to store training and validation losses
    train_losses = []
    val_losses = []
    
    # iterating model training for each epoch
    for epoch in track(range(epochs), description='Training...'):
        model.train()
        train_loss = 0
        train_rmse = 0
        train_rsq = 0
        count = 0
        
        for batch in train_dataloader:
            # set to zero so gradients are not accumulated
            optimizer.zero_grad()
            
            # load images, time series data, and targets onto device
            imgs = batch['img'].to(device)
            ts = batch['ts'].to(device)
            targets = batch['target'].to(device)
            
            # pass images and time series data through model
            out = model(imgs, ts)
            
            # calculate loss and backpropagate through network
            loss = criterion(out, targets.view(-1, 1))
            loss.backward()
            optimizer.step()
            
            # add train_loss for particular training step to epoch training loss
            train_loss += loss.item() * targets.size(0)
            
            # get predicted and actual values for targets
            predicted = out.cpu().detach().numpy()
            actual = targets.cpu().detach().numpy()
            
            # use predicted and actual values for target to calculate rmse and r^2 score
            train_rmse += mean_squared_error(actual, predicted, squared=False)
            train_rsq  += r2_score(actual, predicted)
            count += 1
        
        # divide train_loss by number of training steps in order to get epoch train_loss
        train_loss /= len(train_dataloader.sampler)
        train_losses.append(train_loss)
        
        # divide rmse and r^2 to get epoch rmse and r^2 score
        train_rmse /= count
        train_rsq /= count
        
        # Validation
        model.eval()
        val_loss = 0
        val_rmse = 0
        val_rsq = 0
        count = 0
        
        for batch in val_dataloader:
            optimizer.zero_grad()
            
            imgs = batch['img'].to(device)
            ts = batch['ts'].to(device)
            targets = batch['target'].to(device)
            
            out = model(imgs, ts)
            loss = criterion(out, targets.view(-1, 1))
            
            val_loss += loss.item() * targets.size(0)
            
            predicted = out.cpu().detach().numpy()
            actual = targets.cpu().detach().numpy()
            
            val_rmse += mean_squared_error(actual, predicted, squared=False)
            val_rsq  += r2_score(actual, predicted)
            count += 1
        
        val_loss /= len(val_dataloader.sampler)
        val_losses.append(val_loss)
        
        val_rmse /= count
        val_rsq /= count
        
        # print metrics
        print(
            "\n",
            "\n",
            f"Epoch {epoch+1}/{epochs}:\n",
            f"Train loss: {train_loss:.3f}...\n",
            f"Valid loss: {val_loss:.3f}...\n",
            "\n",
            f"Train RMSE: {train_rmse:.3f}...\n",
            f"Valid RMSE: {val_rmse:.3f}...\n",
            "\n",
            f"Train R^2: {train_rsq}...\n",
            f"Valid R^2: {val_rsq}...\n",
        )
    
    
    # Test
    model.eval()
    
    test_rmse = 0
    test_rsq = 0
    count = 0

#     for batch in test_dataloader:

#         imgs = batch['img'].to(device)
#         targets = batch['target'].to(device)

#         out = model(imgs, targets)
    
#         predicted = out.cpu().detach().numpy()
#         actual = targets.cpu().detach().numpy()
        
#         test_rmse += mean_squared_error(actual, predicted, squared=False)
#         test_rsq  += r2_score(actual, predicted)
#         count += 1

#     test_rmse /= count
#     test_rsq /= count

    print(
        "\n",
        "\n",
        "\n",
        "Training Finished!\n",
#         f"Predictions vs Actual: {submission}",
    )
    
    # display loss curves
    print('Loss Curves: ')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    # plot train losses per epoch
    plt.plot(list(range(epochs)), train_losses, label='train')
    # plot validation losses per epoch
    plt.plot(list(range(epochs)), val_losses, label='valid')
    plt.legend()
    plt.show()

In [ ]:
# randomly sample certain patients to be in the train set and others to be in validation
rng = np.random.default_rng()
num_patients = train_ts['Patient'].nunique()
val_idxs = rng.choice(num_patients, size=math.floor(num_patients * 0.3), replace=False)
train = train_ts.loc[~train_ts['Patient'].isin(val_idxs)].copy()
valid = train_ts.loc[train_ts['Patient'].isin(val_idxs)].copy()

In [ ]:
print(train.shape)
print(valid.shape)

In [ ]:
# instantiate datasets and dataloaders with a batch size of 64
train_dataset = CombinedDataset(train)
valid_dataset = CombinedDataset(valid)

train_dataloader = DataLoader(train_dataset, batch_size=64)
valid_dataloader = DataLoader(train_dataset, batch_size=64)

In [ ]:
# create new efficientnet b3 model and convert model to work with double tensors
net = CombinedModel(32, eff_name='b3')
net.double();

In [ ]:
# run training loop
train_loop(net, train_dataloader, valid_dataloader, epochs=30)

# Cleanup

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()